<h1> Document Vectorization </h1>

This experiment will be based on Google News pretrained Word2Vec and GloVe. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import spacy
import en_core_web_md
import string 
import gensim.downloader
import math
import requests
from sklearn.preprocessing import StandardScaler
import time
import os
import json
import ast

In [35]:
NUM_OF_DOCUMENTS_TRAIN = 1000
NUM_OF_DOCUMENTS_TEST = 500
URL = "https://www.courtlistener.com/api/rest/v3/opinions/"


In [3]:
def get_document(file_name):
    data = ""
    with open(file_name) as json_file:
        data = json.load(json_file)
    return data["plain_text"].replace("\n", " ")


In [36]:
df = pd.DataFrame(columns = ["id", "document"])

i = 0
for file_name in [file for file in os.listdir("data/train/") if file.endswith('.json')]:
    try:
        document = get_document("data/train/" + file_name)
        df.loc[i] = [file_name, document.lower()]
        i += 1
    except Exception as e:
        continue

    

In [37]:
df_test = pd.DataFrame(columns = ["id", "document"])
i = 0
for file_name in [file for file in os.listdir("data/test/") if file.endswith('.json')]:
    try:
        document = get_document("data/test/" + file_name)
        df_test.loc[i] = [file_name, document.lower()]
        i += 1
    except Exception as e:
        continue

In [7]:
df.to_csv("train_pretrained.csv", sep='\t')

In [8]:
df_test.to_csv("test_pretrained.csv", sep='\t')

In [9]:
df

,Unnamed: 0,id,document
0,0,174995.json,united states court of appeals ...
1,1,174996.json,united states court of appeals ...
2,2,175074.json,united states court of appeals ...
3,3,175075.json,united states court of appeals ...
4,4,175076.json,united states court of appeals ...
...,...,...,...
1391,1391,198335.json,united states court of appeals\r ...
1392,1392,198336.json,united states court of appeals\r ...
1393,1393,198337.json,[not for publication--not to be cited as prec...
1394,1394,198338.json,[not for publication--not to be cited as prece...


<h2> Google News Word2Vec </h2>

In [10]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

<h2> GloVe </h2>

In [11]:
glove = gensim.downloader.load('glove-wiki-gigaword-300')

In [12]:
def get_other_vars(measure, avg, wordEmb):
    if measure == cosine_similarity:
        strtype, asc = "cosine similarity", -1
    elif measure == euclidean_distances:
        strtype, asc = "euclidean distance", 1
    else:
        strtype, asc = "manhattan distance", 1
        
    if (avg == True) and (wordEmb == word2vec):
        dftrain = df_avg
    elif (avg == True) and (wordEmb == glove):
        dftrain = df_avg_glove
    elif (avg == False) and (wordEmb == glove):
        dftrain = df_sum_glove
    else:
        dftrain = df_sum
    return strtype, asc, dftrain

<h3> Function for fitting training sets for average and sum of vectors </h3>

In [33]:
def fit_training(model):
    df_sum = pd.DataFrame()
    df_avg = pd.DataFrame()
    
    for idx, row in df.iterrows():
        document = row['document'].translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
        words = document.split()
        word_vec = np.zeros((300, ))
        for word in words:
            if word.lower() in model.key_to_index :
                word_vec += model[word.lower()]
            elif word in model.key_to_index :
                word_vec += model[word]
        if len(words) > 0:
            word_vec_avg = word_vec / len(words)
        else:
            word_vec_avg = 0
        word_vec = pd.Series(word_vec)
        df_sum = df_sum.append(pd.Series(word_vec), ignore_index=True)
        df_avg = df_avg.append(pd.Series(word_vec_avg), ignore_index=True)
    return df_sum, df_avg

In [14]:
def get_idx(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    
    for word in words:
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]
        elif word in model.key_to_index:
            word_vec += model[word]
            
    if avg == True:
        word_vec = word_vec/len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)
    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

In [15]:
def dict_to_df(dict, csv_name):
    df_to_save = pd.DataFrame(dict.items())
    df_to_save.to_csv(csv_name + ".csv", sep='\t')

def df_to_dict(csv_name):
    data_frame = pd.read_csv(csv_name, sep = '\t')
    dict = data_frame.set_index('0').T.to_dict('list')
    return dict

<h3>  Inverse document frequency - IDF </h3>

In [16]:
idf = df_to_dict("idf.csv")

C:\Users\Admin\AppData\Local\Temp/ipykernel_12568/1652140756.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict = data_frame.set_index('0').T.to_dict('list')


In [17]:
# idf = {}
# for i in range(len(df.iloc[:, 1])):
#     tokens = df.iloc[i, 1].translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip().split()
#     for w in tokens:
#         try:
#             idf[w.lower()].add(i)
#         except:
#             idf[w.lower()] = {i}

# for key, value in idf2.items():
#     idf[key] = math.log(len(df.iloc[:, 0]) / (len(value)+1))

In [18]:
def get_idx_idf(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        idfCoeff = 1
        if word.lower() in idf:
            idfCoeff = idf[word.lower()][1]
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*idfCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*idfCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3>  Part-of-speech - POS </h3>

In [19]:
nlp = spacy.load("en_core_web_md")

In [20]:
pos = df_to_dict("pos.csv")

C:\Users\Admin\AppData\Local\Temp/ipykernel_12568/1652140756.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict = data_frame.set_index('0').T.to_dict('list')


In [21]:
# pos = {}

# for sentence in df.iloc[:, 1]:
#     for token in nlp(sentence.lower()):
#         pos[token.text] = {token.pos_ : None}
        
# for sentence in df_test.iloc[:, 1]:
#     for token in nlp(sentence.lower()):
#         pos[token.text] = {token.pos_ : None}

In [22]:
# for word, posTagAndVal in pos.items():
    
#     for tag in posTagAndVal:
#         if tag == "NOUN":
#             posTagAndVal[tag] = 0.7
#         elif tag == "PROPN": 
#             posTagAndVal[tag] = 0.9
#         elif tag == "ADJ":
#             posTagAndVal[tag] = 0.8
#         elif tag == "ADV":
#             posTagAndVal[tag] = 0.7
#         elif tag == "VERB":
#             posTagAndVal[tag] = 0.6
#         elif tag == "SPACE" or tag == "PUNCT":
#             posTagAndVal[tag] = 0
#         else:
#             posTagAndVal[tag] = 0.4

In [23]:
def get_idx_pos(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        posCoeff = 1
        if word.lower() in pos:
            posCoeff = next(iter(ast.literal_eval(pos[word.lower()][1]).values()))
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*posCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*posCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3>   Named Entity Recognition - NER </h3>

In [24]:
ner = df_to_dict("ner.csv")

In [25]:
# ner = {}

# for sentence in df.iloc[:, 1]:
#     for token in nlp(sentence.lower()).ents:
#         ner[token.text] = {token.label_: None}
        
# for sentence in df_test.iloc[:, 1]:
#     for token in nlp(sentence.lower()).ents:
#         ner[token.text] = {token.label_: None}

In [26]:
# for word, nerTagAndVal in ner.items():
    
#     for tag in nerTagAndVal:
#         if (tag == "LOCATION") or (tag == "ORG") or (tag == "NORP") or (tag == "MONEY") or (tag == "WORK_OF_ART") or (tag == "LAW"):
#             nerTagAndVal[tag] = 1.75
#         elif (tag == "GPE") or (tag == "DATE") or (tag == "PERSON") or (tag == "FAC"): 
#             nerTagAndVal[tag] = 1.5
#         elif (tag == "ORDINAL") or (tag == "CARDINAL") or (tag == "PRODUCT") or (tag == "PERCENT") or (tag == "TIME"):
#             nerTagAndVal[tag] = 1.2
#         else:
#             nerTagAndVal[tag] = 1

In [27]:
def get_idx_ner(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        nerCoeff = 1
        if word.lower() in ner:
            nerCoeff = next(iter(ast.literal_eval(ner[word.lower()][1]).values()))
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*nerCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*nerCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype 

In [28]:
def get_idx_pos_ner(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        posCoeff = 1
        nerCoeff = 1
        if word.lower() in ner:
            nerCoeff = next(iter(ast.literal_eval(ner[word.lower()][1]).values()))
        if word.lower() in pos:
            posCoeff = next(iter(ast.literal_eval(pos[word.lower()][1]).values()))
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*nerCoeff*posCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*nerCoeff*posCoeff
            
    if avg == True:
        word_vec = word_vec/len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

In [29]:
def get_idx_pos_idf(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        idfCoeff = 1
        posCoeff = 1
        if word.lower() in pos:
            posCoeff = next(iter(ast.literal_eval(pos[word.lower()][1]).values()))
        if word.lower() in idf:
            idfCoeff = idf[word.lower()][1]
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*posCoeff*idfCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*posCoeff*idfCoeff
            
    if avg == True:
        word_vec = word_vec/len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

In [30]:
def get_idx_ner_idf(type, n, document, model, avg):
    df_q = pd.DataFrame()
    document = document.translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        idfCoeff = 1
        nerCoeff = 1
        if word.lower() in ner:
            nerCoeff = next(iter(ast.literal_eval(ner[word.lower()][1]).values()))
        if word.lower() in idf:
            idfCoeff = idf[word.lower()][1]
        if word.lower() in model.key_to_index:
            word_vec += model[word.lower()]*nerCoeff*idfCoeff
        elif word in model.key_to_index:
            word_vec += model[word]*nerCoeff*idfCoeff
            
    if avg == True:
        word_vec = word_vec/len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = get_other_vars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3> The Experiment </h3>

In [31]:
wordEmbs = [word2vec, glove]
functions = [get_idx, get_idx_idf, get_idx_pos, get_idx_ner, 
             get_idx_pos_ner, get_idx_pos_idf, get_idx_ner_idf]
avgs = [True, False]
measures = [cosine_similarity]


In [38]:
df_sum, df_avg = fit_training(word2vec)

id                                                174995.json
document                  united states court of appeals  ...
Name: 0, dtype: object
id                                                174996.json
document              united states court of appeals      ...
Name: 1, dtype: object
id                                                175074.json
document                 united states court of appeals   ...
Name: 2, dtype: object
id                                                175075.json
document              united states court of appeals      ...
Name: 3, dtype: object
id                                                175076.json
document              united states court of appeals      ...
Name: 4, dtype: object
id                                                175121.json
document              united states court of appeals      ...
Name: 5, dtype: object
id                                                194833.json
document                      united states court of app

id                                                194892.json
document                      united states court of appea...
Name: 65, dtype: object
id                                                194893.json
document    august 26, 1993   united states court of appea...
Name: 66, dtype: object
id                                                194894.json
document    august 26, 1993   united states court of appea...
Name: 67, dtype: object
id                                                194895.json
document    august 26, 1993   united states court of appea...
Name: 68, dtype: object
id                                                194896.json
document                    united states court of appeals...
Name: 69, dtype: object
id                                                194897.json
document                      united states court of appea...
Name: 70, dtype: object
id                                                194898.json
document                      united states court 

id                                                194952.json
document    <p>usage: wpd2html [option] <wordperfect docum...
Name: 125, dtype: object
id                                                194953.json
document      september 28, 1993                     [not ...
Name: 126, dtype: object
id                                                194954.json
document      september 28, 1993                   united ...
Name: 127, dtype: object
id                                                194955.json
document    september 28, 1993    [not for publication]   ...
Name: 128, dtype: object
id                                                194956.json
document                      united states court of appea...
Name: 129, dtype: object
id                                                194957.json
document                      united states court of appea...
Name: 130, dtype: object
id                                                194958.json
document    s   e   p   t   e   m   b   e   

id                                                195008.json
document    november 5, 1993      [not for publication]   ...
Name: 181, dtype: object
id                                                195009.json
document    november 5, 1993                     [not for ...
Name: 182, dtype: object
id                                                195010.json
document                      united states court of appea...
Name: 183, dtype: object
id                                                195378.json
document    may 20, 1994          [not for publication]   ...
Name: 184, dtype: object
id                                                195379.json
document                      united states court of appea...
Name: 185, dtype: object
id                                                195380.json
document                    united states court of appeals...
Name: 186, dtype: object
id                                                195381.json
document    may 23, 1994          [not for p

id                                                195434.json
document    june 20, 1994                     [not for pub...
Name: 240, dtype: object
id                                                195435.json
document                   united states court of appeals ...
Name: 241, dtype: object
id                                                195436.json
document    june 16, 1994     united states court of appea...
Name: 242, dtype: object
id                                                195437.json
document    june 17, 1994         [not for publication]   ...
Name: 243, dtype: object
id                                                195438.json
document    june 21, 1994     united states court of appea...
Name: 244, dtype: object
id                                                195439.json
document    june 21, 1994     united states court of appea...
Name: 245, dtype: object
id                                                195440.json
document                      united states 

...

id                                                196389.json
document      november 20, 1995\r                 united s...
Name: 1150, dtype: object
id                                                196390.json
document     november 8, 1995      [not for publication]\r...
Name: 1151, dtype: object
id                                                196391.json
document     november 8, 1995        [not for publication]...
Name: 1152, dtype: object
id                                                196392.json
document      november 9, 1995      [not for publication]\...
Name: 1153, dtype: object
id                                                196393.json
document      november 17, 1995     [not for publication]\...
Name: 1154, dtype: object
id                                                196394.json
document      november 9, 1995      [not for publication]\...
Name: 1155, dtype: object
id                                                196395.json
document      november 9, 1995        

id                                                196449.json
document     december 4, 1995        [not for publication]...
Name: 1210, dtype: object
id                                                196450.json
document                        united states court of app...
Name: 1211, dtype: object
id                                                196451.json
document                        united states court of app...
Name: 1212, dtype: object
id                                                196452.json
document                        united states court of app...
Name: 1213, dtype: object
id                                                196453.json
document     december 5, 1995        [not for publication]...
Name: 1214, dtype: object
id                                                196454.json
document      december 6, 1995      [not for publication]\...
Name: 1215, dtype: object
id                                                196455.json
document                        united

id                                                196504.json
document                        united states court of app...
Name: 1265, dtype: object
id                                                196505.json
document     january 2, 1996         [not for publication]...
Name: 1266, dtype: object
id                                                196506.json
document      february 8, 1996  united states court of app...
Name: 1267, dtype: object
id                                                196507.json
document                      united states court of appea...
Name: 1268, dtype: object
id                                                196508.json
document                        united states court of app...
Name: 1269, dtype: object
id                                                196509.json
document                      united states court of appea...
Name: 1270, dtype: object
id                                                196510.json
document     january 9, 1996         [

id                                                196571.json
document      march 8, 1996     united states court of app...
Name: 1332, dtype: object
id                                                196572.json
document                        united states court of app...
Name: 1333, dtype: object
id                                                196573.json
document      february 13, 1996     [not for publication]\...
Name: 1334, dtype: object
id                                                196574.json
document      february 13, 1996       [not for publication...
Name: 1335, dtype: object
id                                                196575.json
document     february 13, 1996       [not for publication]...
Name: 1336, dtype: object
id                                                196576.json
document                        united states court of app...
Name: 1337, dtype: object
id                                                196577.json
document      february 14, 1996\r     

id                                                198337.json
document     [not for publication--not to be cited as prec...
Name: 1393, dtype: object
id                                                198338.json
document    [not for publication--not to be cited as prece...
Name: 1394, dtype: object
id                                                198339.json
document                 united states court of appeals\r ...
Name: 1395, dtype: object


In [39]:
df_sum_glove, df_avg_glove = fit_training(glove)

id                                                174995.json
document                  united states court of appeals  ...
Name: 0, dtype: object
id                                                174996.json
document              united states court of appeals      ...
Name: 1, dtype: object
id                                                175074.json
document                 united states court of appeals   ...
Name: 2, dtype: object
id                                                175075.json
document              united states court of appeals      ...
Name: 3, dtype: object
id                                                175076.json
document              united states court of appeals      ...
Name: 4, dtype: object
id                                                175121.json
document              united states court of appeals      ...
Name: 5, dtype: object
id                                                194833.json
document                      united states court of app

id          194891.json
document               
Name: 64, dtype: object
id                                                194892.json
document                      united states court of appea...
Name: 65, dtype: object
id                                                194893.json
document    august 26, 1993   united states court of appea...
Name: 66, dtype: object
id                                                194894.json
document    august 26, 1993   united states court of appea...
Name: 67, dtype: object
id                                                194895.json
document    august 26, 1993   united states court of appea...
Name: 68, dtype: object
id                                                194896.json
document                    united states court of appeals...
Name: 69, dtype: object
id                                                194897.json
document                      united states court of appea...
Name: 70, dtype: object
id                                      

id                                                194960.json
document    september 30, 1993      [not for publication] ...
Name: 133, dtype: object
id                                                194961.json
document                    united states court of appeals...
Name: 134, dtype: object
id                                                194962.json
document    october 4, 1993   united states court of appea...
Name: 135, dtype: object
id                                                194963.json
document    october 4, 1993   united states court of appea...
Name: 136, dtype: object
id                                                194964.json
document                      united states court of appea...
Name: 137, dtype: object
id                                                194965.json
document                    united states court of appeals...
Name: 138, dtype: object
id                                                194966.json
document                    united states co

id                                                195387.json
document    may 27, 1994            [not for publication] ...
Name: 193, dtype: object
id                                                195388.json
document    may 27, 1994                     [not for publ...
Name: 194, dtype: object
id                                                195389.json
document                      united states court of appea...
Name: 195, dtype: object
id                                                195390.json
document    may 27, 1994            [not for publication] ...
Name: 196, dtype: object
id                                                195391.json
document                    united states court of appeals...
Name: 197, dtype: object
id                                                195392.json
document    may 31, 1994                     [not for publ...
Name: 198, dtype: object
id                                                195393.json
document                      united states 

id                                                195442.json
document    june 22, 1994         [not for publication]  u...
Name: 248, dtype: object
id                                                195443.json
document                      united states court of appea...
Name: 249, dtype: object
id                                                195444.json
document                    united states court of appeals...
Name: 250, dtype: object
id                                                195445.json
document                      united states court of appea...
Name: 251, dtype: object
id                                                195446.json
document     [for copies of opinion with appendix, contact...
Name: 252, dtype: object
id                                                195447.json
document    june 23, 1994     united states court of appea...
Name: 253, dtype: object
id                                                195448.json
document    june 27, 1994                   

...

id                                                196390.json
document     november 8, 1995      [not for publication]\r...
Name: 1151, dtype: object
id                                                196391.json
document     november 8, 1995        [not for publication]...
Name: 1152, dtype: object
id                                                196392.json
document      november 9, 1995      [not for publication]\...
Name: 1153, dtype: object
id                                                196393.json
document      november 17, 1995     [not for publication]\...
Name: 1154, dtype: object
id                                                196394.json
document      november 9, 1995      [not for publication]\...
Name: 1155, dtype: object
id                                                196395.json
document      november 9, 1995        [not for publication...
Name: 1156, dtype: object
id                                                196396.json
document                      united s

id                                                196447.json
document                      united states court of appea...
Name: 1208, dtype: object
id                                                196448.json
document     december 4, 1995        [not for publication]...
Name: 1209, dtype: object
id                                                196449.json
document     december 4, 1995        [not for publication]...
Name: 1210, dtype: object
id                                                196450.json
document                        united states court of app...
Name: 1211, dtype: object
id                                                196451.json
document                        united states court of app...
Name: 1212, dtype: object
id                                                196452.json
document                        united states court of app...
Name: 1213, dtype: object
id                                                196453.json
document     december 5, 1995        [

id                                                196504.json
document                        united states court of app...
Name: 1265, dtype: object
id                                                196505.json
document     january 2, 1996         [not for publication]...
Name: 1266, dtype: object
id                                                196506.json
document      february 8, 1996  united states court of app...
Name: 1267, dtype: object
id                                                196507.json
document                      united states court of appea...
Name: 1268, dtype: object
id                                                196508.json
document                        united states court of app...
Name: 1269, dtype: object
id                                                196509.json
document                      united states court of appea...
Name: 1270, dtype: object
id                                                196510.json
document     january 9, 1996         [

id                                                196568.json
document                        united states court of app...
Name: 1329, dtype: object
id                                                196569.json
document                    united states court of appeals...
Name: 1330, dtype: object
id                                                196570.json
document                      united states court of appea...
Name: 1331, dtype: object
id                                                196571.json
document      march 8, 1996     united states court of app...
Name: 1332, dtype: object
id                                                196572.json
document                        united states court of app...
Name: 1333, dtype: object
id                                                196573.json
document      february 13, 1996     [not for publication]\...
Name: 1334, dtype: object
id                                                196574.json
document      february 13, 1996       

id                                                198331.json
document                 united states court of appeals\r ...
Name: 1387, dtype: object
id                                                198332.json
document                united states court of appeals\r  ...
Name: 1388, dtype: object
id                                                198333.json
document                    united states court of appeals...
Name: 1389, dtype: object
id                                                198334.json
document                united states court of appeals\r  ...
Name: 1390, dtype: object
id                                                198335.json
document                 united states court of appeals\r ...
Name: 1391, dtype: object
id                                                198336.json
document                united states court of appeals\r  ...
Name: 1392, dtype: object
id                                                198337.json
document     [not for publication--not

In [40]:
def get_rank(type, n, model, avg, function):
    results = pd.DataFrame(columns = ["verdict", "indexes"])
    indexes = []
    name = ""
    for i, document in enumerate(df_test.iloc[:, 1]):
        indexes, strtype = function(type, n, document, model, avg)
        results = results.append(
            { "verdict" : df_test.iloc[i, 0], 
              "indexes" : indexes}, ignore_index=True)
    return results

In [41]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx)
results.to_csv("results/w2v_sum.csv", sep = "\t")

KeyboardInterrupt: 

In [ ]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx_idf)
results.to_csv("results/w2v_sum_idf.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx_pos)
results.to_csv("results/w2v_sum_pos.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx_ner)
results.to_csv("results/w2v_sum_ner.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx_pos_ner)
results.to_csv("results/w2v_sum_pos_ner.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, word2vec, False, get_idx_ner_idf)
results.to_csv("results/w2v_sum_ner_idf.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx)
results.to_csv("results/glove_sum.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx_idf)
results.to_csv("results/glove_sum_idf.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx_pos)
results.to_csv("results/glove_sum_pos.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx_ner)
results.to_csv("results/glove_sum_ner.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx_pos_ner)
results.to_csv("results/glove_sum_pos_ner.csv", sep = "\t")

In [ ]:
results = get_rank(cosine_similarity, 100, glove, False, get_idx_ner_idf)
results.to_csv("results/glove_sum_ner_idf.csv", sep = "\t")

In [84]:
def gridSearch():
    i = 0
    name = ""
    for wordEmb in wordEmbs:
        for function in functions:
            for avg in avgs:
                for measure in measures:
                    indexes = get_rank(measure, 100, wordEmb, avg, function)
                    break